In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from math import pi

from bokeh.io import show, output_file
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [2]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  16438
Number of unique ingredients:  7325
Number of total ingredients:  545640


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [6]:
ings.head(3)

,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Prods
Prods df is a database of all the products

In [3]:
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


In [7]:
prod_ing.head(3)

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3


In [4]:
#cleanup of a rogue ingredient
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_df.tail()

,id,ingredient
19858,19922,"[Water, Dimethicone, Glycerin, Stearic Acid, P..."
19859,19923,"[Water, Dipropylene Glycol, Caprylic/Capric Tr..."
19860,19924,"[Water, Polyvinyl Alcohol, Ethanolamine, 1,2-H..."
19861,19925,"[Ricinus Communis (Castor) Seed Oil, Lanolin, ..."
19862,19926,"[Water, Ascorbic Acid (Vitamin C, Glycerin, Di..."


In [222]:
products_and_ingredients= pd.merge(prod_ing_df, prods, on='id')
products_and_ingredients.head()

,id,ingredient,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,...,1,0,1,0,1,0,0,1,1,0
2,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,...,0,0,1,1,0,0,1,1,1,0
3,4,"[Petrolatum, Glycerin, Cocamidopropyl Betaine,...",Daily Facials Gentle Clean 4-in-1 Water Activa...,Olay,32.25,4.9,17,0,0,0,...,1,1,0,0,1,0,0,1,1,0
4,5,"[Water, Butylene Glycol, Glycerin, Disodium Su...",Gokujyun Lotion Refill Pouch,Hada Labo,13.26,4.9,55,0,0,0,...,1,0,0,0,1,0,0,1,1,0


In [223]:
ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
ings = ings.drop_duplicates(subset = 'ingredient')
ings.head(3)

,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# Reformatting product + ingredients